# Image classification problem using cnn on 
## MNIST dataset

In [2]:
# image classification problem using cnn
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
# define cnn parameters
learning_rate = 0.001
training_iterations = 150000
batch_size = 128
display_step = 10

In [4]:
# each shape is 28 * 28 array of pixels
n_input = 784
# total classes for classifier
n_classes = 10
# dropout technique solving over fitting issues drop out units (hidden , input and output) in a neural network random
dropout = 0.75

In [5]:
# placeholder for the graph
x = tf.placeholder(tf.float32, [None, n_input])
# change the form of 4D input image to tensor
_X = tf.reshape(x, shape=[-1, 28, 28, 1])

In [6]:
# output foreach digit
y = tf.placeholder(tf.float32, [None, n_classes])

# first layer init weights and bias
wc1 = tf.Variable(tf.random_normal([5, 5, 1, 32]))
bc1 = tf.Variable(tf.random_normal([32]))

Instructions for updating:
Colocations handled automatically by placer.


In [7]:
# 2 Layers
def conv2d(img , weight, bias):
    return tf.nn.relu(tf.nn.bias_add(
                tf.nn.conv2d(img, weight,
                             strides=[1, 1, 1, 1],
                             padding='SAME'), bias))


def max_pool(img , k):
    return tf.nn.max_pool(img,
                          ksize=[1, k, k, 1],
                          strides=[1, k, k, 1],
                          padding='SAME')

In [8]:
# convolution layer
conv1 = conv2d(_X, wc1, bc1)
conv1 = max_pool(conv1, k=2)


# reduce over fitting using dropout
keep_prob = tf.placeholder(tf.float32)
conv1 = tf.nn.dropout(conv1, keep_prob)


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [9]:
# second layer
wc2 = tf.Variable(tf.random_normal([5, 5, 32, 64]))
bc2 = tf.Variable(tf.random_normal([64]))

conv2 = conv2d(conv1, wc2, bc2)
conv2 = max_pool(conv2, k=2)
conv2 = tf.nn.dropout(conv2, keep_prob)

In [11]:
# densely connected layer
wd1 = tf.Variable(tf.random_normal([ 7*7*64, 1024]))
bd1 = tf.Variable(tf.random_normal([1024]))

dense1 = tf.reshape(conv2, [-1, wd1.get_shape().as_list()[0]])
dense1 = tf.nn.relu(tf.add(tf.matmul(dense1, wd1), bd1))
dense1 = tf.nn.dropout(dense1, keep_prob)


# readout layer
wout = tf.Variable(tf.random_normal([1024, n_classes]))
bout = tf.Variable(tf.random_normal([n_classes]))

In [13]:
# predicition
pred = tf.add(tf.matmul(dense1, wout), bout)
# training and testing model
# cost function using softmax with crosss entropy

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
# optimize function for cost .
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
# model evaluation
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


In [14]:
# launching sessions
init = tf.global_variables_initializer()
# evaluate graph
with tf.Session() as sess:
    sess.run(init)
    step=1

    while step * batch_size < training_iterations:
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        sess.run(optimizer,
                        feed_dict={
                                    x: batch_xs,
                                    y: batch_ys,
                                    keep_prob: dropout
                                  }
                 )

        if step % display_step == 0:
            # calc accuracy
            acc = sess.run(accuracy,
                            feed_dict={

                                        x: batch_xs,
                                        y: batch_ys,
                                        keep_prob:1

                                        }
                           )

            loss = sess.run(cost, feed_dict={x: batch_xs, y: batch_ys, keep_prob: 1.})
            print ("Iter " + str(step*batch_size) + ", Minibatch Loss= " + "{:.6f}".format(loss) + ", Training Accuracy= " + "{:.5f}".format(acc))
        step += 1
    print ("Optimization Finished!")
    # Calculate accuracy for 256 mnist test images
    print ("Testing Accuracy:", sess.run(accuracy, feed_dict={x: mnist.test.images[:256], y: mnist.test.labels[:256], keep_prob: 1.}))

Iter 1280, Minibatch Loss= 21266.890625, Training Accuracy= 0.32812
Iter 2560, Minibatch Loss= 12188.378906, Training Accuracy= 0.45312
Iter 3840, Minibatch Loss= 10212.470703, Training Accuracy= 0.52344
Iter 5120, Minibatch Loss= 8018.602051, Training Accuracy= 0.64844
Iter 6400, Minibatch Loss= 5663.015137, Training Accuracy= 0.70312
Iter 7680, Minibatch Loss= 6501.530762, Training Accuracy= 0.67188
Iter 8960, Minibatch Loss= 4249.436523, Training Accuracy= 0.73438
Iter 10240, Minibatch Loss= 2929.196289, Training Accuracy= 0.78125
Iter 11520, Minibatch Loss= 2727.930664, Training Accuracy= 0.78906
Iter 12800, Minibatch Loss= 2504.774658, Training Accuracy= 0.80469
Iter 14080, Minibatch Loss= 3044.786377, Training Accuracy= 0.79688
Iter 15360, Minibatch Loss= 4838.988770, Training Accuracy= 0.73438
Iter 16640, Minibatch Loss= 4260.635742, Training Accuracy= 0.75000
Iter 17920, Minibatch Loss= 2053.603760, Training Accuracy= 0.83594
Iter 19200, Minibatch Loss= 2753.665771, Training Ac